## Recurrent neural network with an LSTM unit

In [1]:
import numpy as np
import pandas as pd
import gensim
import sklearn
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Embedding, Input, TimeDistributed, Dropout
from keras.optimizers import RMSprop

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# hyperparameters

B = 64  #  batch size
R = 300 #  rnn size
S = 3   #  max_sequence len
E = 300 #  embedding size

#### reading and preparing data for training and evaluation

In [2]:
# retrieve fastText embeddings
vec_model = gensim.models.fasttext.FastText.load_fasttext_format('fasttext/cc.fi.300.bin')

In [4]:
# generate training and validation examples for the network
def generate(path, line_limit=13000, mode='train'):
    with open(path, 'r', encoding='utf8') as src:
        word_count = 0
        line_number = 0
        while word_count < word_limit:
            x = np.zeros((B, S, E))
            y = np.zeros((B, S, E))
            word_seqs = [None for _ in range(B)]
            lemma_seqs = [None for _ in range(B)]
            word_seq = []
            lemma_seq = []
            x_seq = []
            y_seq = []
            i = 0
            for line in src:
                line_number += 1
                if line_number > line_limit:
                    return        
                if len(x_seq) == S and len(y_seq) == S:
                    x[i] = np.array(x_seq)
                    y[i] = np.array(y_seq)
                    word_seqs[i] = word_seq[:]
                    lemma_seqs[i] = lemma_seq[:] 
                    if mode == 'train':
                        x_seq.pop(0)
                        y_seq.pop(0)
                        word_seq.pop(0)
                        lemma_seq.pop(0)
                    else:
                        x_seq = []
                        y_seq = []
                        word_seq = []
                        lemma_seq = []
                    i += 1
                    if i >= B:
                        yield x, y, word_seqs, lemma_seqs
                        x = np.zeros((B, S, E))
                        y = np.zeros((B, S, E))
                        word_seqs = [None for _ in range(B)]
                        lemma_seqs = [None for _ in range(B)]
                        i = 0
                        word_count += S
                if len(line) > 2 and line[0] != '#':
                    values = line.split()
                    if '-' not in values[0]:
                        try:
                            word = vec_model.wv[values[1]]
                            lemma_vec = vec_model.wv[values[2]]
                        except:
                            continue
                        x_seq.append(word)
                        y_seq.append(lemma_vec)
                        word_seq.append(values[1])
                        lemma_seq.append(values[2])                        
                else:
                    x_seq = []
                    y_seq = []
                    word_seq = []
                    lemma_seq = []                    

In [ ]:
train_set = [(X, Y) for X, Y, _, _ in generate('UD_Finnish-TDT-master/fi_tdt-ud-train.conllu', line_limit=10000)]
dev_batches = [(x, y, w, l) for x, y, w, l in generate('UD_Finnish-TDT-master/fi_tdt-ud-dev.conllu', line_limit=2000, mode='dev')]

#### defining and training the network

In [9]:
M = Sequential()
M.add(LSTM(R, return_sequences=True, batch_input_shape=(B, S, E)))
M.add(Dropout(.3))
M.add(TimeDistributed(Dense(E, activation='linear')))
M.compile(loss='cosine_proximity', optimizer='rmsprop')

In [10]:
for epoch in range(100):
    
    train_loss = 0
    train_batch_c = 0
    for X, Y in train_set:
        train_loss += M.train_on_batch(X, Y)
        train_batch_c += 1
    
    dev_loss = 0
    dev_batch_c = 0
    for X, Y, _, _ in dev_batches:
        dev_loss += M.test_on_batch(X, Y)
        dev_batch_c += 1
        
    print('epoch:', epoch + 1, 
          '\ttrain loss:', train_loss / train_batch_c, '\t', train_batch_c,
          '\tdev loss:', dev_loss / dev_batch_c, '\t', dev_batch_c)
    
    np.random.shuffle(train_set)

/home/tsolak/.local/lib/python3.5/site-packages/ipykernel_launcher.py:71: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/tsolak/.local/lib/python3.5/site-packages/ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


epoch: 1 	train loss: -0.589608727714716 	 103 	dev loss: -0.6847038439341954 	 7
epoch: 2 	train loss: -0.7197162931405224 	 103 	dev loss: -0.7410256522042411 	 7
epoch: 3 	train loss: -0.7558203542116776 	 103 	dev loss: -0.7621196338108608 	 7
epoch: 4 	train loss: -0.7735321261350391 	 103 	dev loss: -0.7742594225066048 	 7
epoch: 5 	train loss: -0.7840066245458659 	 103 	dev loss: -0.781765912260328 	 7
epoch: 6 	train loss: -0.7907525549814539 	 103 	dev loss: -0.7883425269808088 	 7
epoch: 7 	train loss: -0.7957784290452605 	 103 	dev loss: -0.7909412128584725 	 7
epoch: 8 	train loss: -0.8001167560086667 	 103 	dev loss: -0.7967157534190586 	 7
epoch: 9 	train loss: -0.8037623444807183 	 103 	dev loss: -0.7976611256599426 	 7
epoch: 10 	train loss: -0.8066308787725505 	 103 	dev loss: -0.8018088681357247 	 7
epoch: 11 	train loss: -0.8093644814583861 	 103 	dev loss: -0.801391235419682 	 7
epoch: 12 	train loss: -0.8114950130286726 	 103 	dev loss: -0.8070529358727592 	 7
epoc

epoch: 99 	train loss: -0.8677599476379099 	 103 	dev loss: -0.8385147963251386 	 7
epoch: 100 	train loss: -0.8679860193752548 	 103 	dev loss: -0.8373216390609741 	 7


In [11]:
# evaluate on development set
predictions = []
correct = 0
count = 0
for X, Y, W, L in dev_batches:
    pred = M.predict_on_batch(X)
    for i, seq in enumerate(pred):
        for j, pred_y in enumerate(seq):
            
            nearest = vec_model.wv.most_similar(positive=[pred_y], topn=1)[0][0]
            if nearest == L[i][j]:
                correct += 1
            count += 1
            print('w', W[i][j], '\tl', L[i][j], '\tpred', nearest, nearest == L[i][j])

1
w The 	l The 	pred The True
w Garden 	l Garden 	pred Garden True
w Collection 	l Collection 	pred malliaLämmitys False
w Viikonlopun 	l viikon#loppu 	pred viikonloppu False
w pyöritys 	l pyöritys 	pred pyöritys True
w alkoi 	l alkaa 	pred alkaa True
w järjestämällä 	l järjestää 	pred järjestämällä False
w bloggaajabrunssilla 	l bloggaaja#brunssi 	pred hallintopääkaupunki False
w Helsingissä 	l Helsinki 	pred Helsingissä False
w Showroomilla 	l Showroom 	pred Showroom True
w esiteltiin 	l esitellä 	pred esitella False
w uusi 	l uusi 	pred uusi True
w The 	l The 	pred The True
w Garden 	l Garden 	pred Garden True
w Collection 	l Collection 	pred malliaLämmitys False
w ja 	l ja 	pred ja True
w tarjoiltiin 	l tarjoilla 	pred tarjoilla True
w maukasta 	l maukas 	pred maukas True
w aamupalaa 	l aamu#pala 	pred aamupala False
w aamu-unisille 	l aamu#uninen 	pred aamuöinen False
w muotibloggaajille 	l muoti#bloggaja 	pred viestintäyritys False
w Saavuin 	l saapua 	pred tulla False
w siis 	l 

w Aiheena 	l aihe 	pred aiheena False
w saattoi 	l saattaa 	pred saattaa True
w olla 	l olla 	pred olla True
w Jonna 	l Jonna 	pred Jonna True
w yllättävä 	l yllättävä 	pred yllättävä True
w Asplund-ihastus 	l Asplund#ihastus 	pred kylmäpäisyys False
w tai 	l tai 	pred tai True
w sitten 	l sitten 	pred sitten True
w eräät 	l eräs 	pred se False
w raitabikinit 	l raita#bikinit 	pred leipälaukku False
w , 	l , 	pred , True
w jotka 	l joka 	pred joka True
w Lintu 	l Lintu 	pred Lintu True
w bongasi 	l bongata 	pred bongata True
w . 	l . 	pred . True
w Hassua 	l hassu 	pred Hassua False
w , 	l , 	pred , True
w miten 	l miten 	pred miten True
w sitä 	l sitä 	pred se False
w oppii 	l oppia 	pred opetella False
w katsomaan 	l katsoa 	pred katsoa True
w vaatteita 	l vaate 	pred vaate True
w muiden 	l muu 	pred muu True
w bloggaajien 	l bloggaaja 	pred bloggaaja True
w silmin 	l silmä 	pred silmä True
w ja 	l ja 	pred ja True
w arvaamaan 	l arvata 	pred arvaamaan False
w , 	l , 	pred , True
w k

w smoothieita 	l smoothie 	pred hedelmäsmoothie False
w useammin 	l usein 	pred useammin False
w Yhteen 	l yksi 	pred yksi True
w aikaan 	l aika 	pred aika True
w jaksoinkin 	l jaksaa 	pred jaksoinkin False
w tehdä 	l tehdä 	pred tehdä True
w niitä 	l se 	pred se True
w , 	l , 	pred , True
w mutta 	l mutta 	pred mutta True
w sitten 	l sitten 	pred sitten True
w se 	l se 	pred se True
w vähän 	l vähän 	pred vähän True
w unohtui 	l unohtua 	pred unohtui False
w . 	l . 	pred . True
w Saimme 	l saada 	pred saada True
w lähtiessämme 	l lähteä 	pred meikäpoika False
w vielä 	l vielä 	pred vielä True
w mukaamme 	l mukaan 	pred mukaamme False
w ( 	l ( 	pred ( True
w eväiden 	l eväs 	pred eväiden False
w lisäksi 	l lisäksi 	pred lisäksi True
w , 	l , 	pred , True
w kiitos 	l kiitos 	pred kiitos True
w ) 	l ) 	pred ) True
w alennuskupongin 	l alennus#kuponki 	pred henkilöLisämaksu False
w sekä 	l sekä 	pred sekä True
w uudesta 	l uusi 	pred uusi True
w mallistosta 	l mallisto 	pred mallisto True

w rakentamaan 	l rakentaa 	pred rakentaa True
w jo 	l jo 	pred jo True
w toimivan 	l toimia 	pred toimia True
w ajastimen 	l ajastin 	pred ajastin True
w sekä 	l sekä 	pred sekä True
w ohjaamaan 	l ohjata 	pred ohjata True
w Tiedän 	l tietää 	pred tietää True
w myös 	l myös 	pred myös True
w , 	l , 	pred , True
w että 	l että 	pred että True
w on 	l olla 	pred olla True
w olemassa 	l olla 	pred olla True
w ihan 	l ihan 	pred ihan True
w valmiitakin 	l valmis 	pred valmiitakin False
w Arduinoon 	l Arduino 	pred Arduinokortti False
w pohjautuvia 	l pohjautua 	pred sukeutua False
w time-lapse 	l time-lapse 	pred time-lapse True
w dollyja 	l dolly 	pred dollisuus False
w ajastimilla 	l ajastin 	pred kylmähuone False
w , 	l , 	pred , True
w mutta 	l mutta 	pred mutta True
w tarkoituksena 	l tarkoitus 	pred tarkoitus True
w on 	l olla 	pred olla True
w tehdä 	l tehdä 	pred tehdä True
w kaikki 	l kaikki 	pred kaikki True
w alusta 	l alku 	pred alku True
w lähtien 	l lähtien 	pred lähtien True

w toisella 	l toinen 	pred toinen True
w kun 	l kun 	pred kun True
w ei 	l ei 	pred ei True
w tunnu 	l tuntua 	pred tuntua True
w löytyvän 	l löytyä 	pred löytyvän False
w sopivia 	l sopiva 	pred sopia False
w hammasrattaita 	l hammas#ratas 	pred selkäharja False
w välityksen 	l välitys 	pred viestinvälitys False
w muuttamiseksi 	l muuttaminen 	pred muuttaminen True
w mistään 	l mikään 	pred mistään False
w Lukijoita 	l lukija 	pred lukija True
w varmaan 	l varmaan 	pred varmaan True
w kiinnostaa 	l kiinnostaa 	pred kiinnostaa True
w , 	l , 	pred , True
w paljon 	l paljon 	pred paljon True
w tämä 	l tämä 	pred tämä True
w touhu 	l touhu 	pred touhu True
w kustantaa 	l kustantaa 	pred kustantaa True
w . 	l . 	pred . True
w Tämä 	l tämä 	pred tämä True
w prototyyppi 	l proto#tyyppi 	pred tehtävätyyppi False
w on 	l olla 	pred olla True
w maksanut 	l maksaa 	pred maksaa True
w tähän 	l tämä 	pred tämä True
w mennessä 	l mennessä 	pred mennessä True
w noin 	l noin 	pred noin True
w 150 	l 

w löydy 	l löytyä 	pred löydy False
w , 	l , 	pred , True
w sinut 	l sinä 	pred sinut False
w on 	l olla 	pred olla True
w filtteröity 	l filtteröidä 	pred filtteröinti False
w Twitterin 	l Twitter 	pred Twitterin False
w Tilanteen 	l tilanne 	pred tilanne True
w voi 	l voida 	pred voida True
w onneksi 	l onneksi 	pred onneksi True
w Noudata 	l noudattaa 	pred Työlistaa False
w seuraavia 	l seuraava 	pred yksiköllinen False
w ohjeita 	l ohje 	pred viestintämuoto False
w 1. 	l 1. 	pred 2. False
w Mene 	l mennä 	pred laittaa False
w osoitteeseen 	l osoite 	pred kirjelähetys False
w http://support.twitter.com/forms 	l http://support.twitter.com/forms 	pred henkilöllinen False
w ja 	l ja 	pred ja True
w avaa 	l avata 	pred avata True
w uusi 	l uusi 	pred uusi True
w ticket 	l ticket 	pred ticket True
w ( 	l ( 	pred ( True
w = 	l = 	pred = True
w vikailmoitus 	l vika#ilmoitus 	pred selkäesti False
w / 	l / 	pred / True
w 2. 	l 2. 	pred 2. True
w Kirjoita 	l kirjoittaa 	pred kirjoittaa True


In [12]:
correct / count, correct, count 

(0.7507440476190477, 1009, 1344)